In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


# 1. CNN 구성

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(14*14*256, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 2. 레이블 인코더

In [ ]:
import json

with open('/content/gdrive/MyDrive/03. KOREA UNI/2. Deep Learning/프로젝트/label_encoder.json', 'r', encoding='utf-8') as f:
  label_encoder = json.load(f)

print(label_encoder)
inverse_label_encoder = {v: k for k, v in label_encoder.items()}

{'나폴레옹': 0, '노르웨이숲': 1, '래그돌': 2, '러시안블루': 3, '맹크스': 4, '먼치킨': 5, '먼치킨 롱헤어': 6, '메인 쿤': 7, '발리니즈': 8, '벵갈': 9, '브리티쉬 롱헤어': 10, '브리티쉬 숏헤어': 11, '스노우슈': 12, '스코티쉬 스트레이드': 13, '스코티쉬 스트레이드 롱헤어': 14, '스코티쉬 폴드': 15, '스코티쉬 폴드 롱헤어': 16, '시베리안': 17, '시암': 18, '아메리칸 숏헤어': 19, '아메리칸 컬': 20, '아비시니안': 21, '엑조틱 숏헤어': 22, '코랏': 23, '코리안 숏헤어': 24, '터키시 앙고라': 25, '페르시안': 26, '히말라얀': 27}


# 3. 추론

In [ ]:
from torchvision import transforms
def predict(image_path, model, label_encoder, transform, device):
    image = Image.open(image_path).convert('RGB')
    img_t = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t)
        _, pred = torch.max(outputs, 1)
        breed = label_encoder.get(pred.cpu().numpy()[0].item())
    return breed

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

# 모델 초기화 및 weight 로딩
test_image_path = '/content/gdrive/MyDrive/03. KOREA UNI/2. Deep Learning/프로젝트/CAT/ARCH/cat-arch-032947/frame_0_timestamp_0.jpg'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN(28).to(device)
state_dict = torch.load("/content/gdrive/MyDrive/03. KOREA UNI/2. Deep Learning/프로젝트/MODEL/cat_classification_model.pth", map_location=device)
model.load_state_dict(state_dict)
model.eval()

# 예측
predicted_breed = predict(test_image_path, model, inverse_label_encoder , transform, device)
print(f"예측 품종: {predicted_breed}")


예측 품종: 코리안 숏헤어
